In [1]:
import pandas as pd
import numpy as np
import json
import re

In [23]:
#change 1,2
with open("2018-02-23_focus/2018-02-23_focus_group.json", 'r', encoding='UTF-8') as focus_group:
    focus = json.load(focus_group)

In [24]:
with open("krx_code.json", 'r', encoding='UTF-8') as krx:
    krx_code = json.load(krx)

In [25]:
import json
#change 3
with open("2018-02-23_opening_increase.json", 'r', encoding='UTF-8') as oi:
    increase = json.load(oi)

In [26]:
import bs4
import urllib3

def scrape_price_history(code, time):
    '''
    code: string
    time: string, e.g. 201802090910
    '''

    target = "http://finance.naver.com/item/sise_time.nhn?code=" + code + "&thistime=" + time + "01&page=1"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')
    data_list = soup.find_all("tr")[2].find_all("td",class_="num")
    data_dic = {}
    data_dic["price"] = data_list[0].text
    image = data_list[1].find("img")
    if image != None:
        if image["alt"] == "상승":
            data_dic["price_dif"] = data_list[1].text.strip("\n\t")
        else:
            data_dic["price_dif"] = "-" + data_list[1].text.strip("\n\t")
    else:
        data_dic["price_dif"] = 0
    data_dic["sell"] = data_list[2].text
    data_dic["buy"] = data_list[3].text
    data_dic["volume"] = data_list[4].text
    data_dic["variation"] = data_list[5].text
    
    return data_dic

In [27]:
time = []

#change 4,5
prefix = "2018-02-23_focus/discussion_2018-02-23"
for hour in range(9,16):
    for minute in range(0, 6):
        time.append(prefix + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ ":" + str(minute) + "0.json")

In [28]:
del time[41]
del time[40]

In [29]:
df = pd.DataFrame(columns=["code", "name", "time", "price", "price_dif",\
                                        "sell", "buy", "volume", "variation"])

In [30]:
for t in time:
    with open(t, 'r', encoding='UTF-8') as f:
        discussion = json.load(f)
    for d in discussion:
        if d["name"] in increase:
            row = pd.DataFrame(columns=["code", "name", "time", "price", "price_dif",\
                                        "sell", "buy", "volume", "variation"],\
                           data = [[krx_code[d["name"]], d["name"], d["time"], np.nan, np.nan,\
                                   np.nan, np.nan, np.nan, np.nan]])
            df = df.append(row)

FileNotFoundError: [Errno 2] No such file or directory: '2018-02-23_focus/discussion_2018-02-23 09:00.json'

In [20]:
df = df.reset_index()

In [21]:
len(df)

2840

In [22]:
for idx, row in df.iterrows():
    timestamp = row["time"]
    t = re.sub('[ :-]', '', timestamp)
    
    d = scrape_price_history(row["code"], t)
    
    df["price"].iloc[idx] = d["price"]
    df["price_dif"].iloc[idx] = d["price_dif"]
    df["sell"].iloc[idx] = d["sell"]
    df["buy"].iloc[idx] = d["buy"]
    df["volume"].iloc[idx] = d["volume"]
    df["variation"].iloc[idx] = d["variation"]
    
    print(idx)

C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


KeyboardInterrupt: 

In [ ]:
rv = []
rv.append(df.columns.tolist())
for row in df.iterrows():
    rv.append(row[1].tolist())

In [ ]:
'''
import json
json = json.dumps(rv, ensure_ascii=False)
#change 6
f = open("2018-02-23_price_and_everything.json","w", encoding='UTF-8')
f.write(json)
f.close()
'''
